# Cold-temperature KaiABC reaction driven by periodic \[ATP\]/\[ADP\]

The oscillation is abolished below critical temperature (about $18^{\circ} C$) but what'll happen
if one drives it with periodic change in metabolites?

2024/8/23 Remove dye pipetting commands. Will need to add dye before the robot run

2025/1/10 Decreased sample volume from 6ul to 2ul. There isn't much of a point in keeping a "backup volume".
Change sampling interval to 3.5 hours so that the sampling scheme is not in sync with the circadian program

## Initialize the protocol

In [1]:
# current working dir is /var/lib/jupyter/notebooks
import opentrons.execute
from opentrons import protocol_api
import numpy as np
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous, calibrated_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *
import time

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.13")

# home is required
protocol.set_rail_lights(False)
protocol.home()

## Load labware

- The thermocycler is mounted to slot 7 and 10 (can't be changed)
- The Bio-rad/Eppendorf PCR plate sits in the thermocycler (to hold both the reaction and the samples)
- The GEB tip racks are on slot 4 and 5 (closest slots to the thermocycler to minimize moving time)
- The P10 1st gen pipette is on the left

In [2]:
tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '4')

tc_mod = protocol.load_module(module_name='thermocyclerModuleV2')

# Use Eppendorf twintec LoBind plates cat:14-380-052
# it has similar enough dimensions to the Biorad one
plate = tc_mod.load_labware(name='biorad_96_wellplate_200ul_calibrated')

pipette = protocol.load_instrument("p10_single", "left", 
                                   tip_racks = [tip_rack_1])

## Sampling and liquid settings


0 - 12 h Equilibration to cold temp

11.5 h Add ADP

12 - 24 h Equilibrate to low ATP/ADP

23.5 h Buffer exchange to ATP

24 - 36 h Equilibarte to high ATP/ADP

35.5 h Add ADP

36 - 48 h Equilibrate to low ATP/ADP

47.5 h Buffer Exchange to ATP

48 - 60 h Equilibrate to high ATP/ADP

In [3]:
#!!! change to 3 hours in an experimental run !!!
SAMPLE_INT = 3*60    # time interval in minutes

SAMPLE_STEPS = 20    # plus the initial sampling, in total sample SAMPLE_STEPS + 1 times
SAMPLE_VOL = 2    # ul, volume per sample
SAMPLE_OFFSET = 48    # define the starting well: only use the right half of the plate. First sample well is A7
CTRL_OFFSET = SAMPLE_OFFSET + 24

RM_OFFSET = 0.4

RXN_TUBE = plate.wells_by_name()["C3"]
CTRL_TUBE = plate.wells_by_name()["C4"]
# DYE_TUBE = plate.wells_by_name()["A3"]

The below cell calculates minimal volume in well position

As we lose proteins during buffer exchange we want to start with as much protein as a well can hold.

We will use 100 ul for both tubes

In [4]:
safe_vol = 20    # dead volume on the bottom hard to pipette + room for evaporation
rxn_vol = SAMPLE_VOL * (SAMPLE_STEPS + 1) + safe_vol
# dye_vol = SAMPLE_VOL / 3 * (SAMPLE_STEPS + 1) + safe_vol

# dye_vol = int(np.floor(dye_vol))

print(f"Reaction tube = {rxn_vol} ul")
# print(f"Dye tube at A3, {dye_vol} ul")

Reaction tube = 62 ul


To make the reaction mixture, mix 1.5 uM of KaiA, 3.5 uM of KaiB, and 3.5 uM of KaiC in Kai Storage Buffer (refer to Rust Lab Dropbox for recipe. It is a Tris-HCl-based buffer at pH 8 with Mg++) with 1 mM of ATP

Load well A7 to E9, and A10 to E12 (42 wells in total) with 2 ul of SDS samping buffer and 4ul of water

Wipe down the rubber seal with ethanol before the run!

<div class="alert alert-warning">
    <h2>STOP!</h2>
    <br />
    Before you proceed, please check:<br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?<br />
    Remember to add the dye before the robot run!!
</div>

## Instructions

In [5]:
def sample(idx, v_s):
    "Sampling the in vitro clock and seal with oil"
    
    tc_mod.open_lid()

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]
    ctrl_well = plate.wells()[idx + CTRL_OFFSET]

    # Add the dye before the robot run!!
#     # pipette the dye
#     # defaults: delay = 5, touch tip = true, blowout = true
#     transfer_viscous(pipette, protocol, calibrated_viscous(v_s / 3), DYE_TUBE, cur_well, 
#                      disp_height=0)

    # pipette the sample   
    transfer_viscous(pipette, protocol, calibrated_viscous(v_s - RM_OFFSET), RXN_TUBE, cur_well, 
                     asp_height=2, 
                     disp_height=0.5,
                     delay=0, 
                     if_touch_tip=False, 
                     if_mix=True, 
                     if_blowout=True)
    
    transfer_viscous(pipette, protocol, calibrated_viscous(v_s - RM_OFFSET), CTRL_TUBE, ctrl_well, 
                     asp_height=2, 
                     disp_height=0.5,
                     delay=0, 
                     if_touch_tip=False, 
                     if_mix=True, 
                     if_blowout=True)
    
    tc_mod.close_lid()

## Running

In [6]:
tc_mod.close_lid()
tc_mod.set_lid_temperature(temperature=17)
tc_mod.set_block_temperature(temperature=17)

In [7]:
# make the lists of instructions

scheduler.drop()

# sampling
time_vec = np.arange(SAMPLE_STEPS + 1)*SAMPLE_INT
func_vec = [sample]*(SAMPLE_STEPS + 1)

param_vec = [ (i, SAMPLE_VOL) for i in range(SAMPLE_STEPS + 1) ]

str_vec = ["Sample and mix with loading buffer (Driven & Control)"]*(SAMPLE_STEPS + 1)

n_tip_vec = [2]*(SAMPLE_STEPS + 1)
# register
scheduler.cat(time_vec, func_vec, param_vec, str_vec, n_tip_vec)

# show
scheduler.report(unit="hours")

The protocol requires a total of 42 tip(s)

At 0.00 hours, Sample and mix with loading buffer (Driven & Control), with params (0, 2)
At 3.00 hours, Sample and mix with loading buffer (Driven & Control), with params (1, 2)
At 6.00 hours, Sample and mix with loading buffer (Driven & Control), with params (2, 2)
At 9.00 hours, Sample and mix with loading buffer (Driven & Control), with params (3, 2)
At 12.00 hours, Sample and mix with loading buffer (Driven & Control), with params (4, 2)
At 15.00 hours, Sample and mix with loading buffer (Driven & Control), with params (5, 2)
At 18.00 hours, Sample and mix with loading buffer (Driven & Control), with params (6, 2)
At 21.00 hours, Sample and mix with loading buffer (Driven & Control), with params (7, 2)
At 24.00 hours, Sample and mix with loading buffer (Driven & Control), with params (8, 2)
At 27.00 hours, Sample and mix with loading buffer (Driven & Control), with params (9, 2)
At 30.00 hours, Sample and mix with loading buffer (Driven &

In [ ]:
# running
log_fn = "log/20250123_17C_ATP_ADP_driving.log"
scheduler.run(protocol, log_fn)

In [9]:
# finish up

tc_mod.open_lid()
tc_mod.deactivate_lid()
tc_mod.deactivate_block()

In [10]:
tc_mod.close_lid()

'closed'